In [36]:
from __future__ import division

%load_ext autoreload
%autoreload 2


import sys
sys.path.insert(0, '/home/nima/nanograv/custom_libs/enterprise/')
sys.path.insert(0, '/home/nima/nanograv/custom_libs/enterprise_extensions/')
sys.path.insert(0, '/home/nima/nanograv/custom_libs/PTMCMCSampler/')

import numpy as np
import os, glob, json
import matplotlib.pyplot as plt
import pickle
import scipy.linalg as sl
import healpy as hp
import multiprocessing as mp

from enterprise.signals import parameter
from enterprise.signals import signal_base
from enterprise.signals import deterministic_signals
from enterprise.signals import utils
from enterprise import constants as const

from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import sampler as ee_sampler

pklfile = '/home/nima/nanograv/11yr/NANOGrav_11yr_DE436.pickle'

with open(pklfile, 'rb') as f:
    psrs=pickle.load(f)
    
for each in psrs:
    if each.name == "J1909-3744":
        psr = each

noisefile = '/home/nima/nanograv/11yr/noisefiles/noisedict.json'

U,_ = utils.create_quantization_matrix(psr.toas)
eps = 9  # clip first and last N observing epochs
t0min = np.floor(max(U[:,eps] * psr.toas/const.day))
t0max = np.ceil(max(U[:,-eps] * psr.toas/const.day))

ramp_pta = ee_models.model_ramp([psr], upper_limit=False, bayesephem=False,
                          Tmin_bwm=t0min, Tmax_bwm=t0max)

burst_pta = ee_models.model_bwm([psr])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
No noise dictionary provided!...
No noise dictionary provided!...


In [37]:
noisefile = '/home/nima/nanograv/11yr/noisefiles/noisedict.json'
with open(noisefile, 'rb') as nfile:
    setpars = json.load(nfile)
    
ramp_pta.set_default_params(setpars)
burst_pta.set_default_params(setpars)

INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GASP_efac to 0.970672
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_efac to 1.05058
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_efac to 0.983366
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GUPPI_efac to 1.0416
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GASP_log10_equad to -7.4129
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_log10_equad to -8.08194
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_log10_equad to -6.62141
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GUPPI_log10_equad to -7.3337
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GASP_log10_ecorr to -8.39677
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_log10_ecorr to -7.11967
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_log10_ecorr t

In [38]:
print(ramp_pta.params)
print(burst_pta.params)

[J1909-3744_red_noise_gamma:Uniform(pmin=0, pmax=7), J1909-3744_red_noise_log10_A:Uniform(pmin=-20, pmax=-11), ramp_log10_A:Uniform(pmin=-18, pmax=-11), ramp_t0:Uniform(pmin=53839.0, pmax=57347.0)]
[J1909-3744_red_noise_gamma:Uniform(pmin=0, pmax=7), J1909-3744_red_noise_log10_A:Uniform(pmin=-20, pmax=-11), bwm_costheta:Uniform(pmin=-1, pmax=1), bwm_log10_A:Uniform(pmin=-18, pmax=-11), bwm_phi:Uniform(pmin=0, pmax=6.283185307179586), bwm_pol:Uniform(pmin=0, pmax=3.141592653589793), bwm_t0:Uniform(pmin=53292.01663801381, pmax=57386.79333222201)]


In [21]:
x0=np.hstack(p.sample() for p in burst_pta.params)
print(x0)

[ 5.89234696e-01 -1.57181966e+01 -5.41853120e-01 -1.18618548e+01
  2.17931932e+00  9.03009039e-01  5.33686370e+04]


/home/nima/anaconda3/envs/enterprise/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [41]:
for p in ramp_pta.params:
    print(str(p) + '\n')

J1909-3744_red_noise_gamma:Uniform(pmin=0, pmax=7)

J1909-3744_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)

ramp_log10_A:Uniform(pmin=-18, pmax=-11)

ramp_t0:Uniform(pmin=53839.0, pmax=57347.0)



In [ ]:
#let's try to calculate the associated amplitude and see if the ramp_pta
#gives the same answer

pos = psr.pos

costheta = x0[2]
log10_A = x0[3]
phi = x0[4]
pol=x0[5]
t0=x0[6]

h = 10 ** log10_A
theta=np.arccos(costheta)
t0 *= const.day

apc = utils.create_gw_antenna_pattern(pos, theta, phi)
fp, fc = apc[0], apc[1]

pol = np.cos(2 * pol) * fp + np.sin(2 * pol) * fc

print(pol*h)

In [ ]:
for p in ramp_pta.params:
    print(str(p) + '\n')

In [ ]:
gamma = x0[0]
intrinsic_log10_A = x0[1]
ramp_amp = pol*h
ramp_t0 = x0[6]

print(ramp_pta.get_lnlikelihood([gamma, intrinsic_log10_A, np.log10(ramp_amp), ramp_t0]))

# # Below are cells for testing binary search

In [42]:
from __future__ import division

import numpy as np
import os, sys

from enterprise.signals import utils

def find_closest(target, data):
    closest = 0
    for idx in range(len(data)):
        if abs(target - data[idx]) < abs(target - data[closest]):
            closest = idx

    return idx

def find_two_closest(target, data):
    closest = 0
    next_closest = 1
    
    for idx in range(len(data)):
        if abs(target - data[idx]) < abs(target - data[closest]):
            closest = idx
        elif abs(target - data[idx]) < abs(target - data[next_closest]):
            next_closest = idx
        
    return closest, next_closest

In [43]:
params_dict = {}

log10_burst_amplitudes = np.linspace(-20, -12, 80, endpoint=True) #grid points for the burst strain
log10_rn_amps = np.linspace(-20, -12, 80, endpoint=True) #grid points for the pulsar red noise

Ngammas = 70
gmin = 0
gmax = 7
gammas = np.linspace(gmin, gmax, Ngammas, endpoint=True) #grid points for gamma


U,_ = utils.create_quantization_matrix(psr.toas)
eps = 9  # clip first and last N observing epochs
t0min = np.floor(max(U[:,eps] * psr.toas/const.day))
t0max = np.ceil(max(U[:,-eps] * psr.toas/const.day))

Ts = np.linspace(t0min, t0max, num=100, endpoint=True)

params_dict["J1909-3744_red_noise_gamma"] = gammas
params_dict["J1909-3744_red_noise_log10_A"] = log10_rn_amps
params_dict["ramp_log10_A"] = log10_burst_amplitudes
params_dict["ramp_t0"] = Ts

x0=np.hstack(p.sample() for p in ramp_pta.params)

gamma = x0[0]
log10_rn = x0[1]
log10_ramp = x0[2]
ramp_t0 = x0[3]

print(x0)

[ 6.09247993e+00 -1.19139668e+01 -1.29288996e+01  5.68718524e+04]


/home/nima/anaconda3/envs/enterprise/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [30]:
for each in psrs:
    if each.name == "J1909-3744":
        psr = each.name
        
params_dict = {}

log10_burst_amplitudes = np.linspace(-20, -12, 80, endpoint=True) #grid points for the burst strain
log10_rn_amps = np.linspace(-20, -12, 80, endpoint=True) #grid points for the pulsar red noise

Ngammas = 70
gmin = 0
gmax = 7
gammas = np.linspace(gmin, gmax, Ngammas, endpoint=True) #grid points for gamma


U,_ = utils.create_quantization_matrix(psr.toas)
eps = 9  # clip first and last N observing epochs
t0min = np.floor(max(U[:,eps] * psr.toas/const.day))
t0max = np.ceil(max(U[:,-eps] * psr.toas/const.day))

Ts = np.linspace(t0min, t0max, num=100, endpoint=True)

params_dict["J1909-3744_red_noise_gamma"] = gammas
params_dict["J1909-3744_red_noise_log10_A"] = log10_rn_amps
params_dict["ramp_log10_A"] = log10_burst_amplitudes
params_dict["ramp_t0"] = Ts

## For this search, if we know which indices are closest, we can just
## go directly to the right line

# Find the indices of the closest and next closest numbers to the ne in sample


In [32]:
x0=np.hstack(p.sample() for p in ramp_pta.params)

gamma = x0[0]
log10_rn = x0[1]
log10_ramp = x0[2]
ramp_t0 = x0[3]

print(x0)

[ 8.66722206e-01 -1.71658495e+01 -1.47019379e+01  5.58109006e+04]


/home/nima/anaconda3/envs/enterprise/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [46]:
gamma_closest, gamma_second_closest = find_two_closest(gamma, gammas)
log10rn_closest, log10rn_second_closest = find_two_closest(log10_rn, log10_rn_amps)
log10ramp_closest, log10ramp_second_closest = find_two_closest(log10_ramp, log10_burst_amplitudes)
t0_closest, t0_second_closest = find_two_closest(ramp_t0, Ts)

print(gamma_closest, gamma_second_closest)
print(gamma, gammas[60], gammas[61])

60 61
6.09247993266427 6.086956521739131 6.188405797101449


In [34]:
closest_idx = 

'B1855+09'